# More fun with `pandas`

Let's use `pandas` to dive into some more complicated data.

### The data

We're going to be working with FDA import refusal data from 2014 to September 2017. [From the source](https://www.accessdata.fda.gov/scripts/importrefusals/):

>The Food, Drug, and Cosmetic Act (the Act) authorizes FDA to detain a regulated product that appears to be out of compliance with the Act. The FDA district office will then issue a "Notice of FDA Action" specifying the nature of the violation to the owner or consignee. The owner or consignee is entitled to an informal hearing in order to provide testimony regarding the admissibility of the product. If the owner fails to submit evidence that the product is in compliance or fails to submit a plan to bring the product into compliance, FDA will issue another "Notice of FDA Action" refusing admission to the product. The product then has to be exported or destroyed within 90 days.

Here's the layout for the main file:

Column | Description
------ | -----------
MFG_FIRM_FEI_NUM |
LGL_NAME | Name of the Declared Manufacturer
LINE1_ADRS | Manufacturer Address
LINE2_ADRS | Manufacturer Address
CITY_NAME | Manufacturer City
PROVINCE_STATE | Manufacturer Province or State
ISO_CNTRY_CODE | 2 Letter ISO country code
PRODUCT_CODE | 5-7 Character product code
REFUSAL_DATE |
DISTRICT | FDA District where entry was made
ENTRY_NUM | CBP Entry Number
RFRNC_DOC_ID | CBP Line Number
LINE_NUM | FDA Line number
LINE_SFX_ID | FDA Line Suffix
FDA_SAMPLE_ANALYSIS | Y if there are FDA Analytical Results
PRIVATE_LAB_ANALYSIS | Y if there was a Private Lab package
REFUSAL_CHARGES | asc_id’s (1 to many) of the charges for which product was refused. If there are multiple they will be separated by a comma e.g. 320, 328, 321, 482, 218, 3320
PROD_CODE_DESC_TEXT | FDA's or Corrected Description


### Come up with a list of questions to ask

As with any tool, your analysis is only as good as your questions. We'll start with a couple easy ones:

- In this time period, which country had the most imports refused? (`ISO_CNTRY_CODE`)
- Which company had the most? (`MFG_FIRM_FEI_NUM`)
- What was the most common reason for refusing a product? (`REFUSAL_CHARGES`)

Let's get started!

### Import pandas

In [ ]:
import pandas as pd

### Load the data into data frames

We'll use the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) method to read in the data files:

- `data/import-refusal.csv` => the main data file
- `data/import-refusal-charge-codes.csv` => refusal code lookup file
- `data/country-codes.csv` => country code lookup file ([via](https://github.com/datasets/country-codes/blob/master/data/country-codes.csv))

In [ ]:
# to avoid errors with the FDA files, we're going to specify the encoding
# as latin_1, which is common with gov't data, so it's
# a decent educated guess to start with

# main dataframe
df = pd.read_csv('data/import-refusal.csv', encoding='latin_1')

# country code lookup dataframe
cc_df = pd.read_csv('data/country-codes.csv', header=None, names=['name', 'code'])

# refusal code lookup dataframe
# specify that the 'ASC_ID' column comes in as a string
# because later we're going to join on it
code_df = pd.read_csv('data/import-refusal-charge-codes.csv', encoding='latin_1',
                      dtype={'ASC_ID': str})

In [ ]:
# run `.head()` to check the output
df.head()

### Convert the date field to native datetime

We'll use the [`to_datetime()`](http://pandas.pydata.org/pandas-docs/version/0.20/generated/pandas.to_datetime.html) method to convert the `REFUSAL_DATE` column from string to `datetime`. ([Via this S/O answer](https://stackoverflow.com/a/26763793))

Why? Later on we might want to do some time-based analysis.

In [ ]:
# convert the date strings to datetime
df.REFUSAL_DATE = pd.to_datetime(df.REFUSAL_DATE)

# make sure the conversion actually happened
print(df.REFUSAL_DATE.dtype)

In [ ]:
# run `.head()` to check the country code output
cc_df.head()

In [ ]:
# run `.head()` to check the output
code_df.head()

#### _Which country had the most imports refused?_

Since we're just counting up how many times a value appears in a column, we can slice out the `ISO_CNTRY_CODE` column as a series and use the Series method [`value_counts()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html).

Then we can use the [`reset_index()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html) method to turn the Series into a DataFrame and merge with our data frame of country codes.

In [ ]:
cntry_counts = df['ISO_CNTRY_CODE'].value_counts() \
                                   .reset_index() \
                                   .rename(columns={'index': 'cntry_code', 'ISO_CNTRY_CODE': 'count'})

merged_cntry_counts = cntry_counts.merge(cc_df, left_on='cntry_code', right_on='code', how='left')

merged_cntry_counts[['code', 'name', 'count']].head()

Now we have some follow-up questions. (Why, for instance, are we seeing U.S. companies in there?)

#### _Which company had the most import refusals?_

For this one, we're going to use the [`groupby()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) function and group by two columns: `MFG_FIRM_FEI_NUM` and `LGL_NAME`.

Then we're going to use the [`size()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.size.html) method to compute counts for each group.

Then we're going to use [`reset_index()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html) to turn the Series into a data frame.

Then, after renaming the count column, we're going to sort by count descending.

In [ ]:
df.groupby(['MFG_FIRM_FEI_NUM', 'LGL_NAME']).size() \
                                            .reset_index() \
                                            .rename(columns={0: 'count'}) \
                                            .sort_values('count', ascending=False).head()

**Filtering!**

Let's take a closer look at the records of "DreamCon Co. Ltd." That means _filtering_ our data.

We want to include only records where the `MFG_FIRM_FEI_NUM` is 3010460316. In Python, remember, you use two equals signs to test equality. So our filtering condition is: `df['MFG_FIRM_FEI_NUM'] == 3010460316`. This is the expression that we pass to the data frame inside square brackets.

In [ ]:
freq_flier = df[df['MFG_FIRM_FEI_NUM'] == 3010460316]

freq_flier.head()

#### _What was the most common reason for refusing a product?_

This one will be a little trickier to handle, because the cells in the `REFUSAL_CHARGES` column have a variable number of (comma-separated) refusal codes.

So the thinking here is, we need to:
- Unravel that column into a flat list of codes. We're going to use our friend `split` to turn the string into a list
- Using a list comprehension, iterate over that Series and flatten the lists into one Series
- Count the frequency of each code using the `value_counts()` method
- Turn the Series into a dataframe with `reset_index()`
- Join that dataframe with the code lookup dataframe

In [ ]:
r_codes = df.REFUSAL_CHARGES.str.split(', ', expand=False)

code_counts = pd.Series([st for row in r_codes for st in row]).value_counts() \
                                                              .reset_index() \
                                                              .rename(columns={"index": "code", 0: "count"})

merged = code_counts.merge(code_df, left_on='code', right_on='ASC_ID', how='left')

merged_slim = merged[['code', 'ASC_ID', 'CHRG_CODE', 'CHRG_STMNT_TEXT', 'SCTN_NAME', 'count']]

merged_slim.head()

### Exporting to a CSV

Let's say you wanted to take the merged summary data frame we just merged and write it out to CSV? There's a really simple method for that: [`to_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html). We're going to specify `index=False` because we don't need the pandas-generated index in the first column.

In [ ]:
merged_slim.to_csv('merged-refusal-data.csv', index=False)

### _Exercises_

Let's come up with some more reporting questions and explore them together.